In [68]:
from federation.generate_scenarios import query_time_series_data, fill_null_remove_outliers, generate_scenarios
from schema.schema import MarketPrice
DB_CACHE_FILE = r"../.cache/interim/time_series_schema.db"
data = query_time_series_data(DB_CACHE_FILE, market="FCR", tables=[MarketPrice])["MarketPrice"]
data = fill_null_remove_outliers(data, d_time="1h", z_score=10)
data_scenarios1 = generate_scenarios(data)
data = query_time_series_data(DB_CACHE_FILE, market="RR-pos", tables=[MarketPrice])["MarketPrice"]
data = fill_null_remove_outliers(data, d_time="1h", z_score=10)
data_scenarios2 = generate_scenarios(data)
data = query_time_series_data(DB_CACHE_FILE, market="RR-neg", tables=[MarketPrice])["MarketPrice"]
data = fill_null_remove_outliers(data, d_time="1h", z_score=10)
data_scenarios3 = generate_scenarios(data)

In [69]:
import polars as pl
arbitrary_year = pl.col("timestamp").dt.strftime("2030-%m-%d %H:%M:%S").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False)
df = data_scenarios1.filter(pl.col("scenario")=="median").with_columns(arbitrary_year).sort("timestamp").select(["timestamp", "value"]).to_pandas().set_index("timestamp", drop=True)# .asfreq('30Min', method = 'ffill')
data_df1 = df[~df.index.duplicated()].asfreq('1H', method = 'ffill')
df = data_scenarios2.filter(pl.col("scenario")=="median").with_columns(arbitrary_year).sort("timestamp").select(["timestamp", "value"]).to_pandas().set_index("timestamp", drop=True)# .asfreq('30Min', method = 'ffill')
data_df2 = df[~df.index.duplicated()].asfreq('1H', method = 'ffill')
df = data_scenarios3.filter(pl.col("scenario")=="median").with_columns(arbitrary_year).sort("timestamp").select(["timestamp", "value"]).to_pandas().set_index("timestamp", drop=True)# .asfreq('30Min', method = 'ffill')
data_df3 = df[~df.index.duplicated()].asfreq('1H', method = 'ffill')

In [70]:
import warnings
import numpy as np
from sktime.forecasting.base import ForecastingHorizon
from pycaret.datasets import get_data
from pycaret.time_series import *
exp1 = TSForecastingExperiment()
exp2 = TSForecastingExperiment()
exp3 = TSForecastingExperiment()
warnings.filterwarnings("ignore")
fh = ForecastingHorizon(values=np.arange(24,48), is_relative=True)
s1 = exp1.setup(
    data=data_df1, fh=fh, fold=12, session_id=41
)
s2 = exp2.setup(
    data=data_df2, fh=fh, fold=12, session_id=42
)
s3 = exp3.setup(
    data=data_df3, fh=fh, fold=12, session_id=43
)

,Description,Value
0,session_id,41
1,Target,value
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(8736, 1)"
5,Transformed data shape,"(8736, 1)"
6,Transformed train set shape,"(8689, 1)"
7,Transformed test set shape,"(24, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Description,Value
0,session_id,42
1,Target,value
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(8736, 1)"
5,Transformed data shape,"(8736, 1)"
6,Transformed train set shape,"(8689, 1)"
7,Transformed test set shape,"(24, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Description,Value
0,session_id,43
1,Target,value
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(8736, 1)"
5,Transformed data shape,"(8736, 1)"
6,Transformed train set shape,"(8689, 1)"
7,Transformed test set shape,"(24, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [71]:
from sktime.forecasting.auto_reg import AutoREG  
autoreg_sktime = AutoREG(lags=[i for i in range(24, 72)], trend="n") 
arima1 = exp1.create_model(autoreg_sktime, fit_kwargs={"fh": "{fh}"})
arima2 = exp2.create_model(autoreg_sktime, fit_kwargs={"fh": "{fh}"})
arima3 = exp3.create_model(autoreg_sktime, fit_kwargs={"fh": "{fh}"})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2030-12-16 01:00,0.6871,0.3620,0.6201,0.9567,0.0701,0.0649,0.9424
1,2030-12-17 01:00,0.9035,0.6575,0.8137,1.7352,0.1044,0.0817,0.8169
2,2030-12-18 01:00,0.6785,0.3906,0.6102,1.0295,0.0489,0.0517,0.9276
3,2030-12-19 01:00,0.9100,0.5059,0.8185,1.3332,0.0730,0.0796,0.8531
4,2030-12-20 01:00,0.8865,0.5724,0.7975,1.5086,0.0849,0.0742,0.8369
5,2030-12-21 01:00,0.8904,0.5822,0.8004,1.5334,0.0937,0.0777,0.8296
6,2030-12-22 01:00,1.2171,0.7939,1.0925,2.0885,0.1040,0.1025,0.7048
7,2030-12-23 01:00,1.8749,1.3387,1.6798,3.5173,0.1052,0.1295,0.1961
8,2030-12-24 01:00,1.3590,0.9281,1.2172,2.4372,0.0852,0.0940,0.6261
9,2030-12-25 01:00,2.4585,1.6655,2.2040,4.3776,0.1105,0.1362,0.2701


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2030-12-16 01:00,0.6935,0.2648,18.5762,19.1816,83659942338694000.0000,2.0000,0.0000
1,2030-12-17 01:00,1.1493,0.4314,30.7746,31.2215,138596477402264272.0000,2.0000,0.0000
2,2030-12-18 01:00,1.1830,0.5671,31.6779,41.0107,49025286531944936.0000,1.8234,-0.0782
3,2030-12-19 01:00,0.0846,0.0349,2.2588,2.5168,10172605651748370.0000,1.9167,0.0000
4,2030-12-20 01:00,0.2395,0.1038,6.3976,7.4936,28812441791175372.0000,2.0000,0.0000
5,2030-12-21 01:00,0.1994,0.0914,5.3219,6.5918,23967695407766988.0000,2.0000,0.0000
6,2030-12-22 01:00,0.0407,0.0185,1.0829,1.3354,4876794306087729.0000,1.9167,0.0000
7,2030-12-23 01:00,0.3506,0.3662,9.3069,26.3238,0.1250,0.2500,-0.1429
8,2030-12-24 01:00,0.4345,0.3924,11.5000,28.1691,0.1667,0.3333,-0.2000
9,2030-12-25 01:00,0.1011,0.0455,2.6723,3.2628,12034913870797782.0000,2.0000,0.0000


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2030-12-16 01:00,2.7792,1.5564,22.1057,29.7880,6140043776239784.0000,1.8867,-0.7496
1,2030-12-17 01:00,3.7226,1.7545,29.8315,33.7011,8890640723676861.0000,1.6081,-1.3936
2,2030-12-18 01:00,1.2972,0.6682,10.4310,12.8547,21217355459979576.0000,1.7826,0.1538
3,2030-12-19 01:00,1.4268,0.6871,11.5079,13.2362,36892921585975056.0000,1.8879,-0.1356
4,2030-12-20 01:00,1.6281,1.0332,13.2240,19.9726,20059095695027164.0000,1.9456,-0.1773
5,2030-12-21 01:00,4.0945,1.8230,33.2604,35.2281,549030591498751.5000,1.8078,-8.0504
6,2030-12-22 01:00,1.4079,0.9939,11.4577,19.2211,6815467548478752.0000,1.9237,-0.1928
7,2030-12-23 01:00,0.7808,0.3875,6.3958,7.5110,12022471757991880.0000,1.2652,-1.7965
8,2030-12-24 01:00,0.5664,0.2617,4.6638,5.0805,20260404473937276.0000,1.7704,-2.0719
9,2030-12-25 01:00,0.2259,0.1055,1.8627,2.0499,8229229624126405.0000,1.9489,-229.9772


In [72]:
data1 = exp1.plot_model(arima1, plot="insample", return_data=True)
data2 = exp2.plot_model(arima2, plot="insample", return_data=True)
data3 = exp3.plot_model(arima3, plot="insample", return_data=True)